#### 문장을 벡터로 변환하기

여기에서는 프리트레인을 마친 BERT 모델에 문장을 입력하여

이를 벡터로 변환하는 실습을 진행합니다.

1. 토크나이저 선언

입력값을 만들려면 토크나이저부터 선언해 두어야 합니다. 

다음 코드를 실행하면 kcbert-base 모델이 쓰는 토크나이저를 선언합니다.

In [1]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained( 
    "beomi/kcbert-base",
    do_lower_case=False,
)

2. 모델 초기화

BERT 모델이 프리트레인할 때 썼던 토크나이저를 그대로 사용해야 벡터 변환에 문제가 없습니다. 

모델을 선언할 때 앞 코드와 똑같은 모델 이름을 적용합니다.

In [2]:
from transformers import BertConfig,BertModel 

pretrained_model_config = BertConfig.from_pretrained( 
    "beomi/kcbert-base"
)

model = BertModel.from_pretrained( 
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

pretrained_model_config에는 BERT 모델을 프리트레인할 때 설정했던 내용이 있습니다.

블록(레이어) 수는 12개, 헤드 수는 12개, 어휘 집합의 크기는 30000개 등 정보를 확인할 수 있습니다.

이러한 설정에 따라 모델 전체를 초기화한 뒤 미리 학습된 kcbert-base를 체크포인트로 읽어들이는 역할을 합니다.

3. 입력값 만들기

In [11]:
sentences=["안녕하세요","하이!","섹스 하고 싶다","개새끼야"]

features=tokenizer( 
    sentences,
    max_length=10, 
    padding="max_length",
    truncation=True,
)

코드를 실행하고 feature의 내용을 확인해 보면 다음과 같습니다.

In [12]:
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0], [2, 23135, 8039, 8975, 3, 0, 0, 0, 0, 0], [2, 26998, 15028, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

input_ids : 4개의 입력 문장 각각에 대해 워드피스 토큰화를 수행한 뒤 이를 토큰 인덱스로 변환한 결과

시작점이 2이고 끝점이 3으로 만들었고, padding은 max length에 맞춰 0을 입력하여 나중에 연산에 이제 쓰이지 않도록 합니다.

토큰 최대 길이를 10으로 설정하고, 

토큰 길이가 이보다 짧으면 0으로, 길면 trucation = true이므로 자르도록 합니다. 

따라서 input_ids는 모두 10입니다.

한편 atention_mask는 패딩이 아닌 토큰이 1, 패딩인 토큰이 0으로 실제 토큰이 자리하는지 아닌지에 대한 

boolean성 정보를 제공합니다. 

token_type_ids

세그먼트 정보로 지금처럼 각각이 1개의 문장으로 구성되어있을 떄에는 0입니다.

#### 4. BERT로 단어/문장 수준 벡터 구하기

In [15]:
import torch

피쳐를 토치 텐서로 변환

In [16]:
features = {k: torch.tensor(v) for k,v in features.items()}

In [17]:
features

{'input_ids': tensor([[    2, 19017,  8482,     3,     0,     0,     0,     0,     0,     0],
         [    2, 15830,     5,     3,     0,     0,     0,     0,     0,     0],
         [    2, 23135,  8039,  8975,     3,     0,     0,     0,     0,     0],
         [    2, 26998, 15028,     3,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}

다음 코드를 실행하여 BERT 모델을 수행합니다.

In [18]:
outputs = model(**features)

이 코드의 실행 결과인 outputs는 BERT 모델의 여러 출력 결과를 한데 묶은 것입니다. 

이는 문장 4개의 입력 토큰 각각에 해당하는 BERT의 마지막 레이어 출력 벡터들입니다.

In [19]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-6.9690e-01, -8.2478e-01,  1.7512e+00,  ..., -3.7320e-01,
           7.3986e-01,  1.1907e+00],
         [-1.4803e+00, -4.3981e-01,  9.4438e-01,  ..., -7.4049e-01,
          -2.1058e-02,  1.3064e+00],
         [-1.4299e+00, -5.0328e-01, -2.0686e-01,  ...,  1.2845e-01,
          -2.6110e-01,  1.6057e+00],
         ...,
         [-1.4406e+00,  3.4313e-01,  1.4043e+00,  ..., -5.6484e-02,
           8.4503e-01, -2.1696e-01],
         [-1.3625e+00, -2.4036e-01,  1.1757e+00,  ...,  8.8759e-01,
          -1.0535e-01,  7.3388e-02],
         [-1.4244e+00,  1.5180e-01,  1.2920e+00,  ...,  2.4498e-02,
           7.5718e-01,  7.9514e-03]],

        [[ 9.3707e-01, -1.4749e+00,  1.7351e+00,  ..., -3.4262e-01,
           8.0505e-01,  4.0307e-01],
         [ 1.6095e+00, -1.7269e+00,  2.7936e+00,  ...,  3.1002e-01,
          -4.7874e-01, -1.2491e+00],
         [ 4.8612e-01, -4.5688e-01,  5.7122e-01,  ..., -1.7690e-01,
           1.

In [21]:
outputs.last_hidden_state.shape

torch.Size([4, 10, 768])

In [22]:
outputs.pooler_output

tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955],
        [-0.7335,  0.3168, -0.0450,  ...,  0.4299,  0.0854, -0.9903],
        [ 0.4405, -0.0262, -0.1344,  ..., -0.1273,  0.0095, -0.9968]],
       grad_fn=<TanhBackward0>)

In [23]:
outputs.pooler_output.shape

torch.Size([4, 768])